In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch

C:\Users\nicoesse\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load synthetic queries
syn_queries = '../02_nps_api_data/synthetic_queries.csv'
synthetic_queries_df = pd.read_csv(syn_queries)

In [3]:
query_column = 'query' 
target_column = 'intent' 

In [5]:
# Train/validation split
train_df, val_df = train_test_split(synthetic_queries_df, test_size=0.2, random_state=42)
train_df = train_df.sample(100)
val_df = val_df.sample(20)
print(len(train_df), len(val_df))



100 20


In [6]:
# Convert DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})

In [7]:
# Initialize tokenizer and model
model_name = 'google/flan-t5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
# Preprocess data

def preprocess_function(examples):
    inputs = [f"{target_column}: {query}" for query in examples[query_column]]
    model_inputs = tokenizer(inputs, max_length=512, padding='max_length', truncation=True)
    labels = tokenizer(examples[target_column], max_length=512, padding='max_length', truncation=True).input_ids
    model_inputs['labels'] = labels
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map: 100%|██████████| 20/20 [00:00<00:00, 2488.61 examples/s]


In [9]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1, #3
    weight_decay=0.01,
    logging_dir='./logs',
)


# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
)

# Train and evaluate
trainer.train()
trainer.evaluate()



                                               
100%|██████████| 13/13 [07:57<00:00, 36.76s/it]


{'eval_loss': 41.44375228881836, 'eval_runtime': 27.6214, 'eval_samples_per_second': 0.724, 'eval_steps_per_second': 0.109, 'epoch': 1.0}
{'train_runtime': 477.9253, 'train_samples_per_second': 0.209, 'train_steps_per_second': 0.027, 'train_loss': 39.83060396634615, 'epoch': 1.0}


100%|██████████| 3/3 [00:16<00:00,  5.66s/it]


{'eval_loss': 41.44375228881836,
 'eval_runtime': 29.3347,
 'eval_samples_per_second': 0.682,
 'eval_steps_per_second': 0.102,
 'epoch': 1.0}

In [12]:
# Save the model
model.save_pretrained('./nps_model_intent')
tokenizer.save_pretrained('./nps_model_intent')

('./nps_model_intent\\tokenizer_config.json',
 './nps_model_intent\\special_tokens_map.json',
 './nps_model_intent\\spiece.model',
 './nps_model_intent\\added_tokens.json')